In [ ]:
%pip install beautifulsoup4 selenium

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import os
import json

In [2]:
HEADLESS = False
USERNAME = "your_osvita.ua-username"
PASSWORD = "your_osvita.ua-password"

In [3]:
def init_driver():
    options = webdriver.ChromeOptions()
    if HEADLESS:
        options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver

In [4]:
driver = init_driver()

In [5]:
new_window = driver.window_handles[0]
driver.switch_to.window(new_window)

In [6]:
root_path = 'https://zno.osvita.ua/'
driver.get(root_path + 'mathematics/')

In [7]:
driver.find_element('class name', 'authorization-block').find_element('tag name', 'a').click()
driver.find_element('id', 'floatField01').send_keys(USERNAME)
driver.find_element('id', 'floatField02').send_keys(PASSWORD)
time.sleep(5)
driver.find_element('id', 'floatField02').submit()

In [ ]:
root_path = 'https://zno.osvita.ua/'
driver.get(root_path + 'mathematics/')
    
time.sleep(2)
    
page_html = driver.page_source
soup = BeautifulSoup(page_html,"html.parser")
test_blocks = soup.findAll('ul', class_='tests-block')
tests = [t for test in test_blocks for t in test.findAll("a")]

links = []

for test in tests:
    link = test.get('href')
    links.append(root_path + link[1:])

In [9]:
len(links)

48

In [10]:
links[:1]

['https://zno.osvita.ua/mathematics/585/']

In [11]:
links[36:]

['https://zno.osvita.ua/mathematics/3/',
 'https://zno.osvita.ua/mathematics/121/',
 'https://zno.osvita.ua/mathematics/127/',
 'https://zno.osvita.ua/mathematics/122/',
 'https://zno.osvita.ua/mathematics/128/',
 'https://zno.osvita.ua/mathematics/123/',
 'https://zno.osvita.ua/mathematics/124/',
 'https://zno.osvita.ua/mathematics/129/',
 'https://zno.osvita.ua/mathematics/131/',
 'https://zno.osvita.ua/mathematics/151/',
 'https://zno.osvita.ua/mathematics/292/',
 'https://zno.osvita.ua/mathematics/296/']

### set chrome window scale to 50-60% before running the cell below

In [ ]:
for link in links[:1]:
    if os.path.isfile('new_tests/math_1/' + "-".join(link.split("/")[-3:-1]) + '.json'):
        print(f"\nSkipped {link} as it's already parsed\n")
        continue
    
    driver.get(link)
    driver.find_elements('class name', 'select-answers-variants')[0].find_elements('tag name', 'td')[0].click()
    driver.find_elements('class name', 'q-final')[0].click()
    time.sleep(2)
    tasks = driver.find_element('id', 'tasks-numbers').find_elements('tag name', 'span')
    
    test_data = {'link': link, 'num_tasks': len(tasks)-1, 'tasks': []}
    
    for idx in range(len(tasks)-1):
        task = driver.find_element('id', 'tasks-numbers').find_elements('tag name', 'span')[idx]
        task.click()
        question_block = driver.find_element('id', 'q'+str(idx+1)).find_elements('class name', 'question')[0]
        question = question_block.text
        answers = driver.find_element('id', 'q'+str(idx+1)).find_elements('class name', 'answers')
        
        try:
            answer_table = driver.find_element('id', 'q'+str(idx+1)).find_elements('class name', 'select-answers-variants')[0]
        except Exception as ex:
            print(f"No answer found for test #{idx} due to {ex}")
            answer_table = None
            
        # uncomment lines below to include detailed solutions (note, it requires premium account)
        #try:
            #comment_button = driver.find_element('id', 'q'+str(idx+1)).find_elements('class name', 'explan_open')[0]
            #comment_button.click()
            #comment = driver.find_element('id', 'q'+str(idx+1)).find_elements('class name', 'explanation')[0].text
        #except:
        #    comment = ""
        comment = ""

        if answer_table is not None:
            answer_rows = answer_table.find_elements('tag name', 'tr')
            vheader = None
            hheader = []
            true_answers = []
            for row in answer_rows:
                if vheader is None:
                    vheader = [h.text for h in row.find_elements('tag name', 'th') if h.text]
                    continue
                hh = row.find_elements('tag name', 'th')
                if len(hh) > 0:
                    hheader.append(hh[0].text)
                tds = row.find_elements('tag name', 'td')
                for tidx, td in enumerate(tds):
                    if len(td.find_elements('class name', "ok")) > 0:
                        true_answers.append(vheader[tidx])
        else:
            vheader = None
            hheader = None
            true_answers = []
            
        output_answers = []
        for answer_block in answers:
            for answer in answer_block.find_elements('class name', 'answer'):
                letter = answer.find_elements('class name', 'marker')[0].text
                text = answer.text
                output_answers.append({'answer': letter, 'text': answer.text[len(letter+'\n'):]})
    
        
        test_data['tasks'].append({
            'task_id': idx,
            'question': question,
            'answers': output_answers,
            'answer_vheader': vheader,
            'answer_hheader': hheader,
            'correct_answer': true_answers,
            'comment': comment,
            'with_photo': False
        })
        
        imgs = question_block.find_elements('tag name', 'img')
        if len(imgs):
            test_data['tasks'][-1]['with_photo'] = True
            test_data['tasks'][-1]['photo_url'] = imgs[0].get_attribute("src")
        
        print(f"task-{idx}", question, 
              '\n', output_answers, '\n', vheader, '\n', hheader, '\n', true_answers, '\n', comment, '\n')
        print("\n\n")
        
        time.sleep(2.5)
    
    with open('new_tests/math_1/' + "-".join(link.split("/")[-3:-1]) + '.json', 'w', encoding='utf-8') as fw:
        json.dump(test_data, fw, indent=4, ensure_ascii=False)

In [13]:
test_data['tasks']

[{'task_id': 0,
  'question': 'На рисунку відображено зміну густини (мкг/м3) дрібнодисперсного пилу в повітрі протягом доби в деякому районі міста. Укажіть із-поміж наведених проміжок часу (год), упродовж якого густина такого пилу в повітрі лише зменшувалася.',
  'answers': [{'answer': 'А', 'text': '[\n2\n;\n 6\n]'},
   {'answer': 'Б', 'text': '[\n8\n;\n 12\n]'},
   {'answer': 'В', 'text': '[\n12\n;\n 14\n]'},
   {'answer': 'Г', 'text': '[\n14\n;\n 16\n]'},
   {'answer': 'Д', 'text': '[\n20\n;\n 24\n]'}],
  'answer_vheader': ['А', 'Б', 'В', 'Г', 'Д'],
  'answer_hheader': [],
  'correct_answer': ['Д'],
  'comment': '',
  'with_photo': True,
  'photo_url': 'https://zno.osvita.ua/doc/images/znotest/283/28354/01-math-1-2024-new-2.png'},
 {'task_id': 1,
  'question': 'Зовнішній кут при вершині\nA\nтрикутника\nA\nB\nC\nдорівнює\n100\n∘\n,\n∠\nC\n=\n20\n∘\n(див. рисунок). Визначте градусну міру кута\nB\n.',
  'answers': [{'answer': 'А', 'text': '100\n∘'},
   {'answer': 'Б', 'text': '90\n∘'},
